# Time zones

In [30]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
from astropy.time import Time
import pytz

timezone = pytz.timezone('US/Pacific')
print(timezone)

def convert_to_local_time(timezone, time):
    '''
    Localize ``time`` to the ``timezone``.
    '''
    if not isinstance(time, Time):
        time = Time(time)

    return timezone.localize(time.datetime)

def convert_to_utc(local_time):
    '''
    Convert localized time ``local_time`` to astropy Time object in UTC
    '''
    return Time(local_time.astimezone(pytz.timezone('UTC')).replace(tzinfo=None))

# Present time in UTC
utc_now = datetime.datetime.utcnow()

# Localize to my local time in Pacific timezone
lt = convert_to_local_time(timezone, utc_now)

# Convert back to UTC in astropy time object
UTC = convert_to_utc(lt)

print('Localized datetime {} \n = astropy time {}'.format(lt, UTC))
print('{}, {}'.format(*map(type, [lt, UTC])))

US/Pacific
Localized datetime 2015-07-07 20:57:04.870455-07:00 
 = astropy time 2015-07-08 03:57:04.870455
<type 'datetime.datetime'>, <class 'astropy.time.core.Time'>
